# Tidy data in Pandas

### Handy hints 

Some Jupyter shortcut keys:

- Esc : get into Command mode (leaves Edit mode)
- Enter : edit a cell (puts you in Edit mode)
- h : see help (see all commands)
- Shift+Enter or Ctrl+Enter : run the code in the cell
- a / b : add new cell above/below
- m : turn current cell into a Markdown cell
- y : turn current cell into a Code cell
- 1-6 : turn current cell into a heading (a type of Markdown cell)

You can get help on Python objects and functions with `help()` or the `?` operator.

## Setup 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This causes Jupyter to display any matplotlib plots directly in the notebook
# It also works for pandas and seaborn, since they use matplotlib to render plots
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# pyplot (as plt) is the module we'll primarily use to instantiate matplotlib plot objects
from matplotlib import pyplot as plt
import seaborn as sns

## Pandas 

You can find the Pandas website and documentation at https://pandas.pydata.org/

Pandas is a popular Python library for handling tabular data. It provides much of the same functionality for Python as do data frames in the R language. 

The fundamental data types in Pandas are a Series, representing a 1D array of data:

In [ ]:
s = pd.Series(["Diego","Jessica","Farah"])
s

and a DataFrame, representing a 2D table of data: 

In [ ]:
df = pd.DataFrame({'Name': ["Diego","Jessica","Farah"],
                   'Age': [34, 27, 50]})
df

We can think of a DataFrame like a spreadsheet or a table in a database; every column represents a variable in the dataset. Each column of a DataFrame has a particular type (ints, floats, datetimes, strings etc) and each column can be treated as a Pandas Series.

Above, we constructed data manually using lists and dicts. For the rest of this workshop, we will work with real data. DataFrames are the natural type to use when reading in tabular data from, for instance, CSV files or Excel files.

Here's example to read in some small datasets which we can use for demo purposes:

In [ ]:
iris = pd.read_csv('iris.csv')
cars = pd.read_csv('mtcars.csv')

## Data 

The data we'll use for this workshop is from the [Long Term Evolution Experiment (LTEE)](http://myxo.css.msu.edu/ecoli/). This experiment has been running for over 30 years and over 50,000 E. coli generations, and is still ongoing. Twelve separate populations of E. coli have been propagated for the life of the experiment. Every 500 generations, each population is cloned and stored, allowing researchers to study evolutionary behaviour over the long term, and to "rewind and replay" alternate evolutionary trajectories by propagating from an earlier clone. 

Several interesting events have occurred during the experiment. Some populations have spontaneously developed hypermutator phenotypes. In addition, around generation 31,000 one population, Ara-3, spontaneously developed a rare and novel Cit+ mutation, giving it the ability to metabolise citrate in the substrate.

There have been many publications from this experiment. A handful:

- [Blount et al 2008: Historical contingency and the evolution of a key innovation in an experimental population of Escherichia coli](https://www.pnas.org/content/105/23/7899) - on the spontaneous development of citrate metabolisation and on potentiating mutations
- [Tenaillon et al 2016: Tempo and mode of genome evolution in a 50,000-generation experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4988878) - various investigations by sequencing and variant-calling over 50,000 generations of clones, including discussion of hypermutator phenotypes and genetic drift vs natural selection.

Sequence data from clones is available, but for this workshop we'll just be using some published tabular data.

A version of this dataset is also used by the [Data Carpentry lessons on Genomics](https://datacarpentry.org/genomics-workshop/).

In the repository you'll find the files:

- ltee_sampleruns.csv : sample and sequencing run metadata for the E. coli clones
- ltee_mutations.csv : analysis output from variant calling on the E. coli clones
- ltee_relative_fitness.tsv : relative fitness values for each population and generation up to generation 10,000
- ltee_cell_size.tsv : cell sizes for each population and generation up to 10,000

Let's read in the sample and run metadata. Pandas has functions for reading in many data types. Try looking at the documentation for `read_csv()` by running `help(pd.read_csv)` or `pd.read_csv?`.

In [ ]:
sampleruns = pd.read_csv('ltee_sampleruns.csv')

In [ ]:
# The number of rows and columns
sampleruns.shape

In [ ]:
# The first few rows
sampleruns.head()

In [ ]:
# Column data types
sampleruns.dtypes

In [ ]:
# Column names
sampleruns.columns

In [ ]:
# Index (i.e. row names)
# In this case we didn't provide an index and rows have simply been numbered for us by Pandas
sampleruns.index

#### Exercise

1. Use `pd.read_csv()` to read the file `ltee_mutations.csv` into a variable called `mutations`.
2. Check the column headings and the number of rows in this dataset, and have a look at the first few rows. Compare the size of the dataset and the variables to `sampleruns`.

In [ ]:
mutations = pd.read_csv('ltee_mutations.csv')

In [ ]:
mutations.head()

## Indexing and slicing 

We can extract a column from the DataFrame by indexing with square brackets, e.g. 

In [ ]:
# Extracting a column
sampleruns['Strain ID']

Firstly, let's set our index (row names) to something more meaningful to make it easier to see what's going on. The Strain ID uniquely identifies each sample, so it is probably a sensible index. We can use `sampleruns.set_index()`, or we can assign to the index directly:

In [ ]:
sampleruns.index = sampleruns['Strain ID']
sampleruns.head()

The two most important ways to extract data from a DataFrame are `loc` and `iloc`. `loc` uses the index and the column names; `iloc` uses the row and column numbers, counting from zero. 

In [ ]:
sampleruns.loc['REL768B', 'Accession']

In [ ]:
# Row 0, column 9
sampleruns.iloc[0,9]

In [ ]:
# Rows 1-3, column 9
sampleruns.iloc[1:4, 9]

In [ ]:
# All columns
sampleruns.loc['REL768B', :]

In [ ]:
# Lists let us specify any set of rows and columns, in any order
sampleruns.loc[['REL768A','REL958A'], ['Read Type', 'Read Length']]

In [ ]:
# We can also use True/False values to perform boolean indexing. 
# Pandas will return the rows/columns matching the True values we pass in.
# This will be useful later for filtering data
iris.loc[0:5, [True, False, True, True, False]]

#### Exercise

1. Set the index of `mutations` to be the same as the "Strain ID" column.
2. Extract the population, generation, and number of total mutations for strain REL11345

In [ ]:
mutations.index = mutations['Strain ID']

In [ ]:
mutations.loc['REL11345', ['Population','Generation', 'Total Mutations']]

## Manipulating Series

A single column of a DataFrame is a Series object. Series have a data type:

In [ ]:
sampleruns['Sequencing Depth'].dtype

Like a DataFrame, a Series has an index. In this case we got our Series from a column of a DataFrame, so it will have the same index:

In [ ]:
sampleruns['Sequencing Depth'].index

There are several convenience functions defined on Series. For instance, we can find the average sequencing depth:

In [ ]:
sampleruns['Sequencing Depth'].mean()

Similarly for numeric variables we have, for instance, `.min()` and `.max()`, `.median()`, `std()`, and `sum()`.

`.describe()` is a convenience function for getting several summary statistics at once:

In [ ]:
sampleruns['Sequencing Depth'].describe()

For non-numeric variable types such as strings and categoricals, we may want to look at the unique values.

In [ ]:
sampleruns['Read Type'].unique()

In [ ]:
sampleruns['Read Type'].value_counts()

Pandas makes use of numpy vectorisation, meaning we can do operations on Series with simple syntax, and it will be efficient to run.

In [ ]:
# Assuming 500 generations takes 75 days
mutations['Days'] = mutations['Generation'] * 0.15
mutations.head()

In [ ]:
(mutations['Total Deleted Base Pairs'] + mutations['Total Inserted Base Pairs']).head()

You might have noticed that the `Analysis Notes` column contains a lot of NaN's. This means "not a number" and represents a missing value - i.e. these cells are empty. 

In [ ]:
sampleruns.head()

We can check which values are missing with `.isnull()`. This converts every value in the DataFrame (or Series) into a boolean True/False value.

In [ ]:
sampleruns.isnull().head()

Adding up booleans will treat `True` as `1` and `False` as `0`. A common approach is to use `sum()` to count how many `True` values there are. So we can count missing values like this:

In [ ]:
sampleruns.isnull().sum()

`sampleruns` had 264 rows, so it looks like there are a few non-empty note cells. We could count this explicitly by taking the logical `not` of our True/False values, i.e. adding up cells where `isnull()` is `False`. For manipulating array-like data, we can't use the `not`, `and` and `or` boolean operators. Instead we need to use the bitwise operators `~`, `&`, and `|`.

In [ ]:
(~sampleruns.isnull()).sum()

Notice that Pandas summed each column. We can use `sum(axis=1)` to override this default and sum each row instead.

## Sorting and filtering 

We can sort on a field, or list of fields, with `.sort_values()`:

In [ ]:
# Get a random example subset
subset = sampleruns.sample(15)
subset

In [ ]:
subset.sort_values('Generation')

In [ ]:
subset.sort_values(['Population','Generation'])

We can filter datasets using boolean indexing. This means that if we use a logical expression produce a boolean Series with a logical expression:

In [ ]:
subset['Population'] == 'Ara+5'

we can then select out only the rows (or sometimes columns) where that logical expression is True:

In [ ]:
subset.loc[subset['Population'] == 'Ara+5', :]

#### Exercise 

1. Filter the `sampleruns` dataset to extract only rows which contain Analysis Notes, i.e. those where this field is not empty.
2. Filter the `mutations` dataset to extract only samples with more than 1500 total mutations. 
3. Sort the resulting data from (2) by the number of "Small Indels". Have a look at the resulting Population and Generation columns.


In [ ]:
sampleruns[ ~sampleruns['Analysis Notes'].isnull() ]

In [ ]:
mutations[mutations['Total Mutations'] > 1500].sort_values('Small Indels')

## Interlude: tidy data

[Link to "untidy data" discussion spreadsheet](https://docs.google.com/spreadsheets/d/1P94oEzgxNzlpvYiento53tZxJwaHYi8gpcdpDouu2jw/edit?usp=sharing)

(Don't look too far into this spreadsheet before we get up to it, as it includes solutions which are a spoiler for the exercise.)

## Reshaping data 

We can reshape data using particular columns, with `melt` and `pivot` or `pivot_table`. We'll have a look at this below.

We can also reshape data using the column names and index, with `stack` and `unstack`. This requires MultiIndexes, which we won't go into today.

Here are two tiny "wide" datasets based on our "untidy" housing data:

In [ ]:
sales_wide1 = pd.read_csv('housing-data-wide1.csv')
sales_wide1

In [ ]:
sales_wide2 = pd.read_csv('housing-data-wide2.csv', parse_dates=['date1','date2'])
sales_wide2

How can we reshape these into tidy form? 

The Pandas `melt` function will do this. For instance:

In [ ]:
sales_wide1.melt()

This has put every variable (i.e. every column) into the new `variable` column. This probably isn't what we want. It's only the price columns that are "wide", the other variables were fine.

In [ ]:
# Retain columns property_id and bedrooms
sales_wide1.melt(id_vars=['property_id','bedrooms'])

This is getting close to what we want. The `variable` column contains the original column names and tells us whether the price we're looking at was from the first or second sale (this may or may not be information we care about). The `value` column contains values in the melted columns, i.e. the actual price. 

Now we technically have long form and have eliminated the duplicated `price` variable; all prices are now in the `value` column. Notice that properties can now appear more than once in the table; conceptually, we have a row per sale rather than a row per property. 

We can tell `melt()` what to call the `variable` and `value` columns:

In [ ]:
sales_tidy = sales_wide1.melt(id_vars=['property_id','bedrooms'], 
                 var_name='sale_number',
                 value_name='price')

We still have empty extra rows where there was no sale in the original table, i.e. rows 3 and 5. We could use `dropna()` to get rid of these. A more generic approach would be to use filtering:

In [ ]:
sales = sales_wide1.melt(id_vars=['property_id','bedrooms'], 
                                     var_name='sale_number',
                                     value_name='price')
sales = sales[~sales['price'].isnull()]
sales

This is pretty good! Now we could run commands like `sales["price"].mean()` and get a sensible answer. We'll also be able to use the data easily to produce plots.

If you want a challenge, think about how you could convert `sales_wide2` to tidy form - it's a fair bit harder.

The inverse operation to `.melt()` is `.pivot()`:

In [ ]:
sales.pivot(index='property_id', columns='sale_number', values='price')

Let's have a look at our two remaining LTEE datasets. 

These two files record measurements of:

- the cell size for every population for the first 10,000 generations, measured every 1000 generations
- the relative fitness for every population for the first 10,000 generations, as measured by the growth rate of the strain relative to a reference strain.

In [ ]:
# These are TSV files, so specify tab as the delimiter
cellsize = pd.read_csv('ltee_cell_size.tsv', sep="\t")
fitness = pd.read_csv('ltee_relative_fitness.tsv', sep="\t")

In [ ]:
cellsize

To make life easier down the track, we'll rename the columns to match the variable and population names used in the `sampleruns` and `mutations` tables.

In [ ]:
cellsize.columns = (['Generation'] + ['Ara-{}'.format(n) for n  in range(1,7)] + 
                    ['Ara+{}'.format(n) for n  in range(1,7)])
cellsize.head()

In [ ]:
fitness.columns = (['Generation'] + ['Ara-{}'.format(n) for n  in range(1,7)] + 
                    ['Ara+{}'.format(n) for n  in range(1,7)])

#### Exercise

The cell size and relative fitness data is in wide form. Convert each one to tidy form. When thinking about which columns are "wide", it may help to aim to match the variables in the `sampleruns` and `mutations` tables.

In [ ]:
cellsize_tidy = cellsize.melt(id_vars='Generation', var_name='Population', value_name='Cell size')
fitness_tidy = fitness.melt(id_vars='Generation', var_name='Population', value_name='Relative fitness')

## Merging data 

We can merge two datasets together by matching corresponding variables.

Our main options are `DataFrame.join()` and `pandas.merge()`. `merge()` is a little more flexible, so we'll demonstrate that.

Recall the `cars` dataset:

In [ ]:
cars.head()

The `origin` column here, which is a number 1-3, is actually intended to represent the country of origin. It's encoded as:

- USA : 1
- Europe : 2
- Japan : 3

Let's make a DataFrame to represent this mapping. We'll add a fourth code for Australia, which doesn't appear in the dataset!

In [ ]:
origin_codes = pd.DataFrame(
    {
        'origin': [1,2,3],
        'origin_country': ['USA','Europe','Japan']
    }
)
origin_codes

We can use `pandas.merge()` to join our `cars` table to our `origin_codes` table using the shared `origin` field, like so:

In [ ]:
cars_withorigin = pd.merge(cars, origin_codes)
cars_withorigin.sample(10)

This "just worked" because Pandas correctly deduced that the identically-named field(s) were the ones to match on. Sometimes we might need to be more verbose. In this case, this accomplishes the same thing as the above:

In [ ]:
# use origin from the left dataframe (cars) and from the right (origin_codes)
# use how="left" (keep all origin values that exist in the left dataframe)
cars_withorigin = pd.merge(cars, origin_codes, left_on='origin', right_on='origin', how="left")

Let's try to merge our `sampleinfo` and `mutations` columns. This time there are three shared fields: 'Strain ID', 'Population', and 'Generation'. In fact only 'Strain ID' is needed to uniquely identify  rows, but we want to specify all matching variables so that Pandas knows to only include each of these variables once in the resulting dataframe.

In [ ]:
# may need sampleruns.index.name = mutations.index.name = 'Index'
ltee = pd.merge(sampleruns, mutations, on=['Strain ID','Population','Generation'])
ltee.columns

In [ ]:
ltee.head()

Now that the metadata on Mutator phenotypes is together with the information on actual mutations, we could try exploring the relationships between these fields. Here are a couple of previews of ways to do this!

In [ ]:
sns.boxplot(data=ltee, x='Mutator', y='Total Mutations')

In [ ]:
ltee.groupby('Mutator')['Total Mutations'].mean()

#### Exercise

Merge the cell size data on to the `ltee` table with `pd.merge()`, using the Generation and Population variables. This is only possible if it's been converted to tidy form first, with variable names and values corresponding to those in `1tee`! You will probably want to set `how="left"` (if the cell size data is on the right), since the `ltee` table contains generations that don't exist in the cell size data, and we don't want to throw these rows away.

Do the same with the relative fitness data.

Have a look at missing and repeated values in the newly-added columns. What do you see?

In [ ]:
ltee_merged1 = pd.merge(ltee, cellsize_tidy, on=['Generation', 'Population'], how="left")
ltee_merged2 = pd.merge(ltee_merged1, fitness_tidy, on=['Generation', 'Population'], how="left")

## Writing to a file

To finish off, let's write out our tidied and merged table to a new file, for future analyses. To write to CSV, we can use the `to_csv` method.

In [ ]:
# Use whatever you named your merged dataframe here!
# Don't keep the index as we still have the Strain ID column
ltee.to_csv('ltee_solution.csv', index=False)

# Matplotlib and Seaborn

### Handy hints 

Some Jupyter shortcut keys:

- Esc : get into Command mode (leaves Edit mode)
- Enter : edit a cell (puts you in Edit mode)
- h : see help (see all commands)
- Shift+Enter or Ctrl+Enter : run the code in the cell
- a / b : add new cell above/below
- m : turn current cell into a Markdown cell
- y : turn current cell into a Code cell
- 1-6 : turn current cell into a heading (a type of Markdown cell)

You can get help on Python objects and functions with `help()` or the `?` operator.

In general, we are using plotting libraries that return objects encapsulating the plot. You can check the type of these returned objects with `type()`. 

Some of the plotting libraries we use need to communicate a lot of data to the browser. Current versions of Jupyter shouldn't have a problem, but if you are running an older version, you may need to launch this notebook with a higher data rate limit: `jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000`

## Setup 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This causes Jupyter to display any matplotlib plots directly in the notebook
# It also works for seaborn, since seaborn uses matplotlib to render plots
%matplotlib inline
from matplotlib import pyplot as plt

Be aware that Seaborn automatically changes Matplotlib's defaults *on import*. Not only your Seaborn plots, but also your Matplotlib plots, will look different once Seaborn is imported. If you don't want this behaviour, you can call `sns.reset_orig()` after import, or `import seaborn.apionly as sns` in the first place.

In [ ]:
import seaborn as sns

## Data 

For this workshop we're going to load in two main datasets:

### LTEE data 

This data is from the [Long Term Evolution Experiment (LTEE)](http://myxo.css.msu.edu/ecoli/). This experiment has been running for over 30 years and over 50,000 E. coli generations, and is still ongoing. Twelve separate populations of E. coli have been propagated for the life of the experiment. Every 500 generations, each population is cloned and stored, allowing researchers to study evolutionary behaviour over the long term, and to "rewind and replay" alternate evolutionary trajectories by propagating from an earlier clone. 

Several interesting events have occurred during the experiment. Some populations have spontaneously developed hypermutator phenotypes. In addition, around generation 31,000 one population, Ara-3, spontaneously developed a rare and novel Cit+ mutation, giving it the ability to metabolise citrate in the substrate.

There have been many publications from this experiment. A handful:

- [Blount et al 2008: Historical contingency and the evolution of a key innovation in an experimental population of Escherichia coli](https://www.pnas.org/content/105/23/7899) - on the spontaneous development of citrate metabolisation and on potentiating mutations
- [Tenaillon et al 2016: Tempo and mode of genome evolution in a 50,000-generation experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4988878) - various investigations by sequencing and variant-calling over 50,000 generations of clones, including discussion of hypermutator phenotypes and genetic drift vs natural selection.

Sequence data from clones is available, but for this workshop we'll just be using some published tabular data.

A version of this dataset is also used by the [Data Carpentry lessons on Genomics](https://datacarpentry.org/genomics-workshop/).

In this lesson we'll use a large flat file containing both sample metadata on each clone, and information on observed mutations in their genomes.

In [ ]:
# If you have done the pandas and data tidying workshop and saved the final file,
# you can try reading that file in here if you prefer
ltee = pd.read_csv('ltee_merged.csv',
                   index_col = 'Strain ID')

In [ ]:
ltee.columns

In [ ]:
ltee.head()

### House sales data 

This data represents house sales in Seattle in 2014 and 2015. We have information on the houses themselves: location, size, quality, view, and whether the house is tagged as 'waterfront' or not. We also have information on the date and price of each sale.

In [ ]:
sales = pd.read_csv("housing-data-10000.csv", 
                    usecols=['id','date','price','lat','long', 'zipcode',
                             'waterfront','view','grade','sqft_living'],
                    parse_dates=['date'], 
                    dtype={'zipcode': 'category',
                           'waterfront': 'bool'})

# We'll downsample in order to make plots a bit simpler and smaller to store
# In particular Altair defaults to a limit of 5000 observations
sales = sales.sample(3000)

In [ ]:
sales.dtypes

In [ ]:
sales.head()

In [ ]:
sales.dtypes

### Demo data 

We'll read in the classic `iris` and `mtcars` datasets for demo purposes.

In [ ]:
iris = pd.read_csv('iris.csv')
cars = pd.read_csv('mtcars.csv')

In [ ]:
iris.head()

In [ ]:
cars.head()

## Matplotlib 

Matplotlib is the oldest and still the fundamental plotting library in Python. It has a huge range of capabilities. Many other libraries (including Seaborn) use Matplotlib as a back-end renderer.

Today we're focussing on plotting tabular data. We won't touch on all Matplotlib's capabilities. If you want to see more of the range of things Matplotlib can do, you can look through the [Matplotlib gallery](https://matplotlib.org/gallery.html.), or try out this excellent [Matplotlib tutorial](https://www.labri.fr/perso/nrougier/teaching/matplotlib/).

Some simple Matplotlib plots, inline in the notebook:

In [ ]:
plt.scatter(x=[1,2,3],y=[5,6,8])

In [ ]:
plt.scatter(iris['petal_length'], iris['petal_width'])

It's better to use the object-oriented Axes methods to draw the plot:

In [ ]:
fig, ax = plt.subplots()

ax.scatter(iris['petal_length'], iris['petal_width'])

In [ ]:
fig

In [ ]:
type(fig)

An example Matplotlib plot with legend and annotation:

In [ ]:
x = [1,2,3,4,5]
y = [2,5,10,17,26]
y2 = [1,4,9,11,9]

fig, ax = plt.subplots()
ax.plot(x, y, c='blue', label='Projected')
ax.scatter(x, y2, c='red', label='Actual')
fig.legend()
ax.annotate("where it all went wrong", 
                                 xy=(3,10), xytext=(1,12),
                                 arrowprops={'width':2})

fig.savefig('example_matplotilb.png')

## Seaborn 

Seaborn builds on Matplotlib. Some nice features are:

- works directly with Pandas dataframes, concise syntax
- lots of plot types, including some more advanced options
- statistical plotting: many plots do summary statistics for you
- good default aesthetics and easy control of aesthetics
- uses Matplotlib, so can use all Matplotlib backends (incl lots of image file formats)
- underlying Matplotlib objects can be tweaked directly

For completeness, here's the plot we made before:

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9]
})

fig, ax = plt.subplots()
sns.scatterplot(data=df, x='Time', y='Actual', color='red', ax=ax)
sns.lineplot(data=df, x='Time', y='Projected', color='blue', ax=ax)

ax.set_ylabel('Huge profits')

ax.annotate("where it all went wrong", 
                                 xy=(3,10), xytext=(1,12),
                                 arrowprops={'width':2})

Notice that we can add changes like annotations in exactly the same way, as we have Matplotlib Figure and Axes objects.

In [ ]:
# Demo: seaborn scatterplot can do hue as just another channel
sns.scatterplot(data=iris, 
                x='petal_length', 
                y='petal_width',
                hue='species',
                size='sepal_width')

### Seaborn and Pandas 

In some cases we can use Seaborn by passing in lists (or arrays or series) directly:

In [ ]:
sns.barplot(x=['A','B','C'], y=[33,44,20])

However Seaborn is aware of Pandas and it is very common to use Seaborn directly with DataFrames. Plotting functions can take a DataFrame as their `data` parameter and then refer directly to column names:

In [ ]:
sns.barplot(data=iris, x='species', y='petal_length')

Here Seaborn has interpreted the `x` and `y` arguments as field names in the supplied DataFrame. Notice also that Seaborn has performed the summary statistics for us - in this case, using the default `estimator`, which is `mean()`. 

Notice also what happens if we simply swap the `x` and `y` parameters. Seaborn will automatically deduce that the categorical or string-like variable must be the bar labels, and the numeric variable must be the numeric axis:

In [ ]:
sns.barplot(data=iris, y='species', x='petal_length')

What happens if the grouping is not obviously categorical? How could we fix this?

In [ ]:
sns.barplot(data=cars, x='acceleration', y='origin')

In [ ]:
cars.dtypes

In [ ]:
# parameter orient="h", or appropriate Pandas dtype
cars['origin'] = cars['origin'].astype('category')

#### Exercises: 

**1:** Create a count plot using `sns.countplot()` on the `ltee` data, showing how many clones have each `Mutator` phenotype. Note that you do not need to specify the `y` axis variable for a countplot, just the `x` axis variable (i.e. category).

**2:** Create a (vertical) bar plot using the `sales` data, showing how house prices vary with the value of the property `grade`.

Bar plots are often deplored as a way of showing statistical estimates, as only the top of the bar is really important, and the bar itself is a visual distraction. A point plot is an alternative, and plots like box plots can show more information. Several other plot types also show distributional information within categories.

**3:** Reproduce the plot you just made, using instead each of the Seaborn functions:

- pointplot()
- boxplot()
- violinplot()  (try the `scale` parameter)
- boxenplot()
- stripplot() [SEE WARNING]  (try the `jitter` parameter)
- swarmplot() [SEE WARNING]

Note what sort of information about the distribution is shown by each.

WARNING: `stripplot()` and `swarmplot()` will plot individual data points. There are too many house sales to easily display in this way - you may want to subsample the dataframe with e.g.  `data=sales.sample(100)`.

In [ ]:
sns.countplot(data=ltee, x='Mutator')

In [ ]:
sns.barplot(data=sales, x='grade', y='price')

In [ ]:
?sns.violinplot

In [ ]:
sns.violinplot(data=sales, x='grade', y='price', scale='width')

### Hue 

Many Seaborn plotting functions take a `hue` parameter. This colours the plot elements by some categorical variable, but more than this, summary statistics are calculated for each level of the hue variable.

In [ ]:
sns.scatterplot(data=ltee, 
                x='Generation', 
                y='Synonymous Base Substitutions', 
                palette='bright', hue='Population')

In [ ]:
sns.lineplot(data=ltee, 
            x='Generation', 
            y='Synonymous Base Substitutions', 
            palette='bright', hue='Population')

In [ ]:
sns.lineplot(data=ltee, 
            x='Generation', 
            y='Relative fitness', 
            palette='bright', hue='Population')

In [ ]:
sns.lmplot(data=sales, x='sqft_living', y='price', hue='waterfront')

#### Exercises:

1. Subset the `ltee` dataset to get only the clones where the `Mutator` value is "PM"
2. Create an `lmplot` of `Total Mutations` over time (i.e. against `Generation`). Do this without a `hue` parameter, then add in `Population` as the `hue` parameter. 
3. Try adding the `hue` parameter to one of your previous plots of some other type - for instance, a box plot.

In [ ]:
subset = ltee[ ltee['Mutator']=="PM" ]

In [ ]:
# or, convenient but less flexible
subset = ltee.query('Mutator=="PM"')

In [ ]:
sns.lmplot(data=subset, x='Generation', y='Total Mutations', 
           hue='Population')

If we'd wanted to colour scatter points by some continuous variable, `hue` can be made to work, but doesn't really make sense as it is intended for discrete values. In this specific case, we could pass our colour variable down to the underlying Matplotlib scatter call via the `scatter_kws` parameter. We'll look more at this later. Or, if we're not trying to fit a linear model, we could just use Seaborn aesthetics around a Matplot scatter plot:

In [ ]:
sample = sales.sample(200)

with sns.axes_style('white'):
    fig, ax = plt.subplots(figsize=(8,8))
    ax.scatter(x=sales['long'], y=sales['lat'], c=sales['price'].apply(np.log), 
               alpha=0.5, s=7, cmap='Reds')

For discrete color palettes, as needed by the `hue` parameter, Seaborn has a `color_palette()` function to generate a useful range of palettes. You can find [a tutorial on choosing color palettes here](https://seaborn.pydata.org/tutorial/color_palettes.html). 

### Compound plots 

Seaborn has some plotting functions which create more complex figures made of multiple subplots. These include `cat()`, `catplot()`, `jointplot()`, `lmplot()` and `clustermap()`. Let's see a few examples:

In [ ]:
# jointplot shows a scatter or density plot, with marginal distributions
sns.jointplot(data=sales, x='sqft_living', y='price') #, kind='reg')

In [ ]:
# pairplot shows pairwise relationships between variables
sns.pairplot(data=ltee,
             vars=['Generation', 'Total Mutations', 'Nonsynonymous Base Substitutions', 'IS Element Insertions'])
             #hue='Mutator')

In [ ]:
# pairplot shows pairwise relationships between variables
sns.pairplot(data=ltee,
             vars=['Generation', 
                   'Total Mutations', 
                   'Nonsynonymous Base Substitutions', 
                   'IS Element Insertions'],
             hue='Mutator')


In [ ]:
# catplot conditions different subplots on different categorical variable values
# we map variables to row and column of a grid of plots (as well as to hue)
# in this example, we just use columns, and so have only one row
sns.catplot(data=cars, kind='box',
               x='year', y='mpg', 
               row='origin',
               aspect=2)

In [ ]:
# vs a single (unfaceted) boxplot
sns.boxplot(data=cars, x='year', y='mpg')

#### Exercise:

Create a joint plot using the `ltee` data showing `Cell size` against `Relative fitness`

In [ ]:
sns.jointplot(data=ltee,
              x='Relative fitness',
              y='Cell size')

#### Exercise: 

Design a plot using `sns.catplot()`, to show the relationship between house price and: grade, view, and whether the property is waterfront. Available channels of information are:

- x and y coordinates
- hue
- row and column of subplot (`row` and `col`)

You can set the `kind` parameter to the kind of plot you want to make: point, bar, count, box, violin, or stripplot.

You do not have to use all of these channels - in fact your plot may be difficult to take in if you do.

### Colour and Palettes

Seaborn has good colour options. There are a few ways we could want to use access colours:

* Specify an individual colour for some plot element. Matplotlib named colours can be used, or rgb values specified. Also check out the `sns.xkcd_rgb` dictionary of 954 named colours from the XKCD colour survey - for instance, `sns.xkcd['fire engine red']` is a colour.
* Specify a colormap, for mapping a continuous value to colour. All Matplotlib colormaps can be used by name. You can see these under the `plt.cm` module. Seaborn's `light_palette()` and `dark_palette()` functions can also generate custom colourmaps easily.
* Specify a discrete colour palette (a list of colours), for mapping a discrete or categorical variable to colour. In Seaborn, there is a distinction between colour palettes and colormaps. In general, you can create a colour palette by explicitly listing some colours, or by selecting a series of colours along some colormap. There are several functions, including `color_palette()`, `light_palette()`, `dark_palette()`, `diverging_palette()` and `xkcd_palette()`, which can produce many discrete colour palettes of whatever size you need. 

In [ ]:
# An example discrete colour palette of 7 colours, based on the XKCD colour "denim blue"
# palplot is a function to visualise a palette
palette = sns.light_palette("denim blue", n_colors=7, input='xkcd')
sns.palplot(palette)

In [ ]:
# Equivalently (to illustrate that we can use an rgb value directly)
denim_blue = sns.xkcd_rgb["denim blue"]
print(denim_blue)
palette = sns.light_palette(denim_blue, n_colors=7)
print(palette)
sns.palplot(palette)

**Exercise:** Try out the Seaborn function `choose_diverging_palette()` in your notebook (it requires no arguments). You can assign the result to a variable.

In [ ]:
p = sns.choose_diverging_palette()

In [ ]:
p

In [ ]:
sns.xkcd_rgb['denim blue']

Let's try a heatmap. Unlike most Seaborn functions, heatmaps take data in wide form. We'll need to pivot our long-form data to get a table of numbers, indexed by two variables. The heatmap function will then transform each number to a colour via a continuous colourmap.

#### Exercise: 

Use `ltee.pivot_table()` to produce a table showing the (average) number of mutations per Population and Generation. The x-axis (column headers) should be the values of the `Generation` variable, and the rows (index) should be the values of the `Population` variable. If you're new to pivot tables in Pandas, check the example below that uses the `cars` dataset.

In [ ]:
mutations_table = ltee.pivot_table(index='Population', 
                                   columns='Generation',
                                   values='Total Mutations',
                                   aggfunc='mean')

In [ ]:
# Here's an example using the toy dataset
# Avoid looking at the details first if you want to solve the above without hints!

speed_table = cars.pivot_table(index='origin', columns='year', values='mpg', aggfunc=np.mean)
speed_table

In [ ]:
sns.heatmap(speed_table, vmin=0, annot=True)

#### Exercises:

* If you haven't already, produce a heatmap with the `ltee` pivot table you produced above. You will probably want to leave off the annotations unless your plot is very large. 
* Specify a different colourmap using the `cmap` parameter to `heatmap`. An ascending (not diverging) colourmap is appropriate for counts that are all positive.
* Some populations have far more mutations than others, and so our heatmap doesn't really show detail for the lower end of the scale. Try to plot a heatmap where colour is based on the *log* of the mutation count.  

In [ ]:
sns.heatmap(np.log(mutations_table), vmin=0, cmap='Greens')

### Seaborn and Matplotlib 

Seaborn plotting functions call Matplotlib plotting functions, and we can pass arguments down to these underlying functions.

For instance, `lmplot()` calls `scatter()` and `plot()` to draw points and lines. We can pass arguments down using `scatter_kws` and `line_kws`:

In [ ]:
sns.lmplot(data=sales, x='grade', y='price',
           scatter_kws={'alpha': 0.3},
           line_kws={'linestyle': 'dashed', 'color': 'red'})

**Exercise:** Which underlying plotting function is called by your `catplot` above? You can try passing arguments to it with catplot's `kwargs`.

Let's look at the object returned by a Seaborn plot function.

In [ ]:
g = sns.countplot(ltee['Mutator'])

In [ ]:
type(g)

This is a Matplotlib `Axes` object. We can use all the usual `Axes` methods to tweak the plot. What's more, if we have an existing `Axes` object, we can ask Seaborn to draw the plot onto it by passing it in as the `ax` parameter:

In [ ]:
fig, myaxes = plt.subplots(figsize=(7,5), facecolor=sns.xkcd_rgb['pale pink'])

sns.countplot(ltee['Mutator'], color='purple', ax=myaxes)

And the plot is still attached to `fig`:

In [ ]:
fig

The simpler Seaborn plotting functions return `Axes` objects, and can take an `Axes` as a parameter. More complex functions like `jointplot()` and `catplot()` need to make multiple subplots. What do they return?

In [ ]:
g = sns.pairplot(data=ltee,
             vars=['Generation', 'Total Mutations', 'Nonsynonymous Base Substitutions', 'IS Element Insertions'],
             hue='Mutator')

In [ ]:
type(g)

In [ ]:
type(g.fig)

In [ ]:
g.axes

In [ ]:
g.fig

As we can see, these more complex functions produce entire Matplotlib `Figure` objects, which can contain multiple `Axes`. The `Figure`, however, comes wrapped in a Seaborn class, which provides some convenient functions to manipulate the figure properties.

In [ ]:
# g.set sets a property on all Axes in the Figure
# set x-axis to log scale:
g.set(xscale='log')
g.fig

The Seaborn classes commonly returned are:

- JointGrid : a central bivariate plot with two marginal univariate plots. Used by `jointplot()`. 
- PairGrid : a grid of subplots for plotting pairwise relationships. Has convenience functions for mapping plots onto diagonal and non-diagonal elements. Used by `pairplot()`.
- FacetGrid : a grid of subplots showing the same relationship, conditioned on some variable across different subplots. Designed to map fields of a Pandas DataFrame to different rows, columns, and hues. Used by `catplot()` and `lmplot()`.

It's possible to instantiate these classes yourself for custom plots.

# Altair

### Handy hints 

Some Jupyter shortcut keys:

- Esc : get into Command mode (leaves Edit mode)
- Enter : edit a cell (puts you in Edit mode)
- h : see help (see all commands)
- Shift+Enter or Ctrl+Enter : run the code in the cell
- a / b : add new cell above/below
- m : turn current cell into a Markdown cell
- y : turn current cell into a Code cell
- 1-6 : turn current cell into a heading (a type of Markdown cell)

You can get help on Python objects and functions with `help()` or the `?` operator.

In general, we are using plotting libraries that return objects encapsulating the plot. You can check the type of these returned objects with `type()`. 

Some of the plotting libraries we use need to communicate a lot of data to the browser. Current versions of Jupyter shouldn't have a problem, but if you are running an older version, you may need to launch this notebook with a higher data rate limit: `jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000`

## Setup 

Altair is not bundled with Anaconda - you may need

`conda install -c conda-forge altair vega`

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# We may want to use some colours etc from other libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import and initialise altair
import altair as alt
# this line is needed in jupyter notebook, but not jupyter-lab
alt.renderers.enable('notebook')

## Data 

For this workshop we're going to load in two main datasets:

### LTEE data 

This data is from the [Long Term Evolution Experiment (LTEE)](http://myxo.css.msu.edu/ecoli/). This experiment has been running for over 30 years and over 50,000 E. coli generations, and is still ongoing. Twelve separate populations of E. coli have been propagated for the life of the experiment. Every 500 generations, each population is cloned and stored, allowing researchers to study evolutionary behaviour over the long term, and to "rewind and replay" alternate evolutionary trajectories by propagating from an earlier clone. 

Several interesting events have occurred during the experiment. Some populations have spontaneously developed hypermutator phenotypes. In addition, around generation 31,000 one population, Ara-3, spontaneously developed a rare and novel Cit+ mutation, giving it the ability to metabolise citrate in the substrate.

There have been many publications from this experiment. A handful:

- [Blount et al 2008: Historical contingency and the evolution of a key innovation in an experimental population of Escherichia coli](https://www.pnas.org/content/105/23/7899) - on the spontaneous development of citrate metabolisation and on potentiating mutations
- [Tenaillon et al 2016: Tempo and mode of genome evolution in a 50,000-generation experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4988878) - various investigations by sequencing and variant-calling over 50,000 generations of clones, including discussion of hypermutator phenotypes and genetic drift vs natural selection.

Sequence data from clones is available, but for this workshop we'll just be using some published tabular data.

A version of this dataset is also used by the [Data Carpentry lessons on Genomics](https://datacarpentry.org/genomics-workshop/).

In this lesson we'll use a large flat file containing both sample metadata on each clone, and information on observed mutations in their genomes.

In [ ]:
# If you have done the pandas and data tidying workshop and saved the final file,
# you can try reading that file in here if you prefer
ltee = pd.read_csv('ltee_merged.csv',
                   index_col = 'Strain ID')

In [ ]:
ltee.head()

### House sales data 

This data represents house sales in Seattle in 2014 and 2015. We have information on the houses themselves: location, size, quality, view, and whether the house is tagged as 'waterfront' or not. We also have information on the date and price of each sale.

In [ ]:
sales = pd.read_csv("housing-data-10000.csv", 
                    usecols=['id','date','price','lat','long', 'zipcode',
                             'waterfront','view','grade','sqft_living'],
                    parse_dates=['date'], 
                    dtype={'zipcode': 'category',
                           'waterfront': 'bool'})

# We'll downsample in order to make plots a bit simpler and smaller to store
# In particular Altair defaults to a limit of 5000 observations
sales = sales.sample(3000)

In [ ]:
sales.head()

### Demo data 

We'll read in the classic `iris` and `mtcars` datasets for demo purposes.

In [ ]:
iris = pd.read_csv('iris.csv')
cars = pd.read_csv('mtcars.csv')

## Altair 

[Altair](https://altair-viz.github.io) is a library for creating interactive plots. 

Altair is built around the [Vega-Lite](https://vega.github.io/vega-lite/) schema, a "visualisation grammar". Altair plots are specified in Python, then converted behind the scenes to a declarative JSON structure that follows the Vega-Lite schema, which can then be rendered by a Javascript library.

Altair works very well with Pandas - in fact, it usually expects data to be in a Pandas DataFrame.

### Simple interactive plotting

Here's a simple example of an Altair plot.

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9]
})

chart = alt.Chart(df)

chart = chart.mark_line(color='blue')

chart = chart.encode(x='Time', y='Projected')

In [ ]:
chart

Here `mark_line()` tells Altair we want to draw a line, and `encode()` is used to assign our variables (columns of the DataFrame) to possible encoding channels of the line. We've just used `x` and `y`. Notice that we set the colour in `mark_line()`; if we'd wanted to set the colour to encode some variable, we would have set it in `encode()`.

Simple pan-and-zoom interactivity can be added with `.interactive()`:

In [ ]:
chart = alt.Chart(df)

chart.mark_line(color='blue').encode(x='Time', y='Projected').interactive()

If we wanted to produce the chart we produced earlier, with scatter points, we can use `alt.layer()`, which takes the components as arguments, or just the `+` operator: 

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9],
})

chart = alt.Chart(df)
line = chart.mark_line(color='blue').encode(x='Time', y='Projected')
points = chart.mark_point(color='red').encode(x='Time',y='Actual')

# or alt.layer(line, points).interactive()
(line + points).interactive()

#### Exercise: 

Recreate, in Altair, the scatter plot of house sales with x-coordinates given by `longitude` and y-coordinates given by `latitude`. You can use `.mark_point()` or `.mark_circle()`. One issue you will discover, if you use the `x` and `y` encodings, is that Altair and Vega-lite include the axis zero by default. 

In [ ]:
# Solution
alt.Chart(data=sales)\
    .mark_circle(size=20)\
    .encode(alt.X('long', 
                  scale=alt.Scale(zero=False)), 
            alt.Y('lat', 
                  scale=alt.Scale(zero=False))) # or e.g. alt.Scale(domain=(45,48))

In [ ]:
# Solution
alt.Chart(data=sales)\
    .mark_circle(size=20)\
    .encode(longitude='long',
            latitude='lat')

## Data types 

Altair recognises four fundamental [data types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types):

| Data Type | Shorthand Code | Description |
|-|-|-|
| quantitative | Q | a continuous real-valued quantity |
| ordinal | O | a discrete ordered quantity |
| nominal | N | a discrete unordered category |
| temporal | T | a time or date value |

In [ ]:
alt.Chart(sales.sample(10))\
    .mark_bar()\
    .encode(x='date:N', y='price')

In [ ]:
alt.Chart(sales.sample(10))\
    .mark_bar()\
    .encode(x='date:T', y='price:Q')

In [ ]:
list(ltee['Read Length'].unique())

In [ ]:
# What kind of sequencing experiments have been done on the LTEE data?

chart = alt.Chart(data=ltee)

chart.mark_point()\
    .encode(x=alt.X('Sequencing Depth', 
                    scale=alt.Scale(type='log')),
            y='Read Type',
            color='Read Length:N')  # try N, O, Q

In [ ]:
# What kind of sequencing experiments have been done on the LTEE data?

chart = alt.Chart(data=ltee)

chart.mark_point()\
    .encode(x=alt.X('Sequencing Depth', 
                    scale=alt.Scale(type='log')),
            y='Read Type',
            color='Read Length:O')

#### Exercise:

* Colour your "geographical" scatter plot of house sales according to whether the property is or is not a waterfront property. What is the appropriate data type?
* Colour according to price (if you prefer, try a log scale). What is the appropriate data type?
* Colour according to view. What is the appropriate data type?

In [ ]:
alt.Chart(sales)\
    .mark_circle(size=10)\
    .encode(y=alt.Y('lat', 
                    scale=alt.Scale(zero=False)),
            x=alt.X('long',
                    scale=alt.Scale(zero=False)),
            color='waterfront:N',
            tooltip='price')\
    .interactive()

In [ ]:
alt.Chart(sales)\
    .mark_circle(size=10)\
    .encode(y=alt.Y('lat', 
                    scale=alt.Scale(zero=False)),
            x=alt.X('long',
                    scale=alt.Scale(zero=False)),
            color=alt.Color('price:Q', 
                            scale=alt.Scale(type='log')),
            tooltip='price')\
    .interactive()

In [ ]:
alt.Chart(sales)\
    .mark_circle(size=10)\
    .encode(y=alt.Y('lat', 
                    scale=alt.Scale(zero=False)),
            x=alt.X('long',
                    scale=alt.Scale(zero=False)),
            color='view:O',
            tooltip='price')\
    .interactive()

## Marks and encodings 

Altair has various marks used to draw different kinds of plots. For instance:

* `mark_point()` : points on a scatter plot
* `mark_bar()` : rectangular bars, in e.g. a bar plot or histogram
* `mark_area()` : filled/shaded areas

The full list of marks can be found at [https://altair-viz.github.io/user_guide/marks.html](https://altair-viz.github.io/user_guide/marks.html).

We can assign variables of the DataFrame to different encoding channels. For instance:

* `x` and `y` : position of mark (scatterpoint position, top of bar-chart bar, etc)
* `color` : colour of the mark (colour of point, bar, shaded area etc)
* `size` : size of the mark (point size, bar width, etc)

You can find a list of encodings at [https://altair-viz.github.io/user_guide/encoding.html](https://altair-viz.github.io/user_guide/encoding.html).

In [ ]:
# Demo: marks from common base
base = alt.Chart(sales.sample(10))\
    .encode(x='date:T', y='price:Q') 

base.mark_point(color='red') | base.mark_bar()

In [ ]:
# Boxplot demo
base = alt.Chart(ltee)

base.mark_boxplot().encode(x='Population', 
                           y=alt.Y('Total Mutations', scale=alt.Scale(type='log')))
# Log scale works here
# Color also works here (sort of!)

In [ ]:
base = alt.Chart(ltee)

base.mark_boxplot()\
    .encode(x='Population:N', 
            y=alt.Y('Total Mutations',
                  scale=alt.Scale(type='log')),
            color='Mutator')
# Note that some info is hidden with colour - not a true grouped boxplot

In [ ]:
pd.crosstab(ltee['Mutator'], ltee['Population'])

In [ ]:
alt.Chart(ltee)\
    .mark_line()\
    .encode(x=alt.X('Generation', scale=alt.Scale(domain=[0,10000])),
            y=alt.Y('Relative fitness', scale=alt.Scale(zero=False)),
            color='Population')


## Aggregations 

Many common plot types involve summarising the data:

* histograms involve putting some continuous variable into bins (_binning_), and then _counting_ how many samples are in each bin
* bar charts or point plots usually involve plotting the _mean_ of some variable, and possibly error bars based on _confidence intervals_
* box plots usually involve finding the _mean_, _quartiles_, _min_ and _max_ of some variable (although as of v3 Altair has a built-in boxplot mark type).

Combining aggregation functions with different kind of marks is very powerful. 

Most aggregations have a shorthand string form (e.g. encode `y='count()'`) and a longer form (e.g encode `y=alt.Y(aggregate='count', type='quantitative')`).

Binning is done using an explicit axis object like `alt.X()` or `alt.Y()`, with `bin=True` or `bin=alt.Bin()`.

You can find a list of aggregation functions at [https://altair-viz.github.io/user_guide/encoding.html#binning-and-aggregation](https://altair-viz.github.io/user_guide/encoding.html#binning-and-aggregation)

In [ ]:
alt.Chart(iris)\
    .mark_bar()\
    .encode(x='species',
            y='mean(petal_length)')

In [ ]:
# Demo: Histogram of iris petal lengths
alt.Chart(iris)\
    .mark_bar()\
    .encode(x=alt.X('petal_length', bin=alt.Bin(step=0.5)),
            y='count()')

In [ ]:
# Demo: LTEE heatmap
chart = alt.Chart(ltee)
chart = chart.mark_rect()\
    .encode(x='Generation:O', 
            y='Population', 
            color=alt.Color('mean(Total Mutations)',
                            scale=alt.Scale(type='log')))

# saving the chart as html
chart.interactive().save('ltee_heatmap.html')

chart

#### Exercise:
1. Using `mark_bar()`, create a histogram of total mutation counts from the `ltee` data.
2. Try the same plot but using `.mark_area()`.
3. On the original histogram, try setting the `color` encoding to the `Mutator` field. Do you get a stacked or layered histogram by default? (This can be controlled using the `stack` parameter to `alt.Y()`).

In [ ]:
chart = alt.Chart(ltee)

chart.mark_bar()\
    .encode(alt.X('Total Mutations', 
                  bin=alt.Bin(maxbins=20)),
            y=alt.Y('count()'))

In [ ]:
chart = alt.Chart(ltee)

chart.mark_area()\
    .encode(alt.X('Total Mutations', 
                  bin=alt.Bin(maxbins=20)),
            y=alt.Y('count()'))

In [ ]:
chart = alt.Chart(ltee)

chart.mark_bar(opacity=0.5)\
    .encode(alt.X('Total Mutations', 
                  bin=alt.Bin(maxbins=20)),
            y=alt.Y('count()', stack=None),
            color='Mutator')

## Interactivity 

Hover text can be added to a plot simply by setting the `tooltip` encoding.

#### Exercise: 

On your scatter plot of house location, set the hover info to display the sale price.

In [ ]:
alt.Chart(sales)\
    .mark_circle(size=10)\
    .encode(y=alt.Y('lat', 
                    scale=alt.Scale(zero=False)),
            x=alt.X('long',
                    scale=alt.Scale(zero=False)),
            tooltip='price')\
    .interactive()

Basic pan-and-zoom interactivity is set with `.interactive()`. This is actually short for:

```
chart.add_selection(
    alt.selection_interval(bind='scales')
)
```

This means we've added a behaviour where dragging the mouse (selecting an interval) causes the plot to zoom and pan (bind='scales'). We can create custom interactive behaviour with [bindings, selections, and conditions](https://altair-viz.github.io/user_guide/interactions.html).


We can define particular kinds of selections with Altair functions:

* `selection_interval()` allows us to select everything in a region of the plot by dragging over it
* `selection_single()` allows us to select a single element by clicking on it
* `selection_multi()` allows us to select multiple elements by holding shift and clicking

`add_selection()` is used to the defined selection object to a plot.

`alt.condition()` is used to change plot encodings conditional on whether an element is currently selected or not. This is what makes the plots interactive.

#### Exercise:

You're given the below plot, where we've added a selection that changes the colour of the points in `area_price_chart` when selected. Edit the `location_chart` so that when points are selected in `area_price_chart`, they are _also_ highlighted in `location_chart`.

In [ ]:
selected_houses = alt.selection_interval()

area_price_chart = alt.Chart(data=sales)\
    .mark_circle(size=20, opacity=0.5)\
    .encode(
        x='sqft_living:Q',
        y=alt.Y('price:Q', scale=alt.Scale(type='log')),
        color=alt.condition(selected_houses,
                            alt.value('red'),
                            alt.value('lightgrey')))\
    .add_selection(selected_houses)

# We could optionally .add_selection to the second chart too
# This would mean points could be selected using either chart
location_chart = alt.Chart(data=sales)\
    .mark_circle(size=20)\
    .encode(
        alt.X('long', scale=alt.Scale(zero=False)),
        alt.Y('lat', scale=alt.Scale(zero=False)),
        color=alt.condition(selected_houses,
                            alt.value('red'),
                            alt.value('lightgrey'))
        )

area_price_chart | location_chart

In [ ]:
# Just colour-on-select example

# Note we need ltee.reset_index()
# Altair needs 'Strain ID' to be a column, not the index, to use it
# Also note selected points are coloured by a variable - Altair's default assumption

selected = alt.selection_interval()

chart = alt.Chart(ltee.reset_index())\
    .mark_circle()\
    .encode(x='Generation',
            y='Total Mutations',
            color=alt.condition(selected,
                                'Population',
                                alt.value('gray')),
            tooltip='Strain ID')\
    .add_selection(selected)

chart